## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [1]:
!git clone https://github.com/Yui-Arthur/WMamba.git > /dev/null
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git > /dev/null

Cloning into 'WMamba'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 184 (delta 16), reused 14 (delta 11), pack-reused 147
Receiving objects: 100% (184/184), 117.89 KiB | 2.18 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/kaggle/working/WMamba
Cloning into 'U-Mamba'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 301 (delta 46), reused 18 (delta 16), pack-reused 231
Receiving objects: 100% (301/301), 2.91 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (74/74), done.


#### Mabma & U-Mamba & WMamba reqirement

In [2]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [3]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [4]:
!sudo bash setup_wmamba.sh

Setup WMamba
CP File From W-Mamba to U-Mamba
Trainer:
  nnUNetTrainerUMambaEncNoAMP.py
  nnUNetTrainerWMambaBase.py
  nnUNetTrainerUMambaEnc.py
  nnUNetTrainerWMambaBot.py
  nnUNetTrainerUMambaBot.py
Nets:
  WMambaBot_3d.py
Others:
  paths.py
  waveletLayer.py


### set env variable

In [5]:
### DO NOT EDIT !!! ###
nnUNet_preprocessed="/home/nnUNet_preprocessed/"
nnUNet_raw="/home/nnUNet_raw/"
nnUNet_results="/kaggle/working/"

%env ipynb=1
%env nnUNet_preprocessed={nnUNet_preprocessed}
%env nnUNet_raw={nnUNet_raw}
%env nnUNet_results={nnUNet_results}



env: ipynb=1
env: nnUNet_preprocessed=/home/nnUNet_preprocessed/
env: nnUNet_raw=/home/nnUNet_raw/
env: nnUNet_results=/kaggle/working/


### parameter (can modify)

In [6]:
### Epoch ###
# **the maximum epoch include all train epoch in every session** #
%env training_epochs=1

### Checkpoint ###
# every {checkpoint_save_every} epochs will save one checkpoint
%env checkpoint_save_every = 10
load_checkpoint = False
prev_checkpoint_f0 = "/kaggle/input/0505_1/checkpoint_latest_0.pth"
prev_checkpoint_f1 = "/kaggle/input/0505_1/checkpoint_latest_1.pth"

### Kaggle Dataset Input Path ###
dataset_path = "/kaggle/input/Task04_Hippocampus.mp4"
### Dataset ID ###
dataset_id = 4

### Set Trainer/Model ###
trainer = "nnUNetTrainerWMambaBot"

### Set GPU DRAM for dataset preproccess ###
gpu_dram = 8


env: training_epochs=1
env: checkpoint_save_every=10


### dataset prepaired

In [7]:
if dataset_id == 110:
    """===== Dataset 110 Covid-19 ===="""
    """ unrar data folder and move to home. """
    !apt install unrar
    !(unrar x {dataset_path} {nnUNet_raw}) >  /dev/null
    """dataset preoproccess about 20 mins."""
    !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 110
    dataset_name = "Dataset110_CTImage"
else:
    """===== Dastaset 4 Hippocampus ===="""
    """ unzip data folder and move to home. """
    !(tar -C  /home/ -xvf {dataset_path}) > /dev/null
    """ dataset preoproccess. """
    !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus 
    !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 4
    dataset_name = "Dataset004_Hippocampus"


100%|█████████████████████████████████████████| 260/260 [00:08<00:00, 31.12it/s]


### train

In [8]:
from pathlib import Path
if not load_checkpoint:
    ### first training (no checkpoint) ###
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --npz
else:
    ### load checkpoint, continue train
    current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"

    !mkdir -p {current_trainer_folder}/fold_0
    !mkdir -p {current_trainer_folder}/fold_1

    !cp {prev_checkpoint_f0} {current_trainer_folder}/fold_0/checkpoint_best.pth
    !cp {prev_checkpoint_f1} {current_trainer_folder}/fold_1/checkpoint_best.pth
    
    # check the checkpoint can be successful loaded
    f0 = [i.name for i in Path(current_trainer_folder+"fold_0").glob("*.pth")]
    f1 = [i.name for i in Path(current_trainer_folder+"fold_1").glob("*.pth")]
    print("Load Checkpoint fold 0",f0)
    print("Load Checkpoint fold 1",f1)
    if len(f0) <1 or len(f1) <1:
        raise ValueError("Load Checkpoint Failed")
    
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --c --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --c  --npz

2024-05-21 11:55:02.554556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 11:55:02.554571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 11:55:02.554620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 11:55:02.554685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 11:55:02.829356: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory

### find best fold

In [9]:
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1


***All results:***
nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres: 0.8478260249721554

*Best*: nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres: 0.8478260249721554

***Determining postprocessing for best model/ensemble***
Results were improved by removing all but the largest foreground region. Mean dice before: 0.84783 after: 0.84784
Removing all but the largest component for 1 did not improve results! Dice before: 0.85376 after: 0.85376
Results were improved by removing all but the largest component for 2. Dice before: 0.84191 after: 0.84192

***Run inference like this:***

nnUNetv2_predict -d Dataset004_Hippocampus -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 1 -tr nnUNetTrainerWMambaBot -c 3d_fullres -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file /kaggle/working/Dataset004_Hippocampus/nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres/crossval_results_folds_0_1/postprocessing.p

### predict 

In [10]:
!nnUNetv2_predict -i "{nnUNet_raw}{dataset_name}/imagesTs"\
-o "{nnUNet_results}{dataset_name}/predTs" \
-d {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 > /dev/null

2024-05-21 12:00:08.031602: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 12:00:08.031665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 12:00:08.033291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/waveletLayer.py:30: UserWarning: The wavelet_level is zero, auto set one to avoid error
  warnings.warn("The wavelet_level is zero, auto set one to avoid error")
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  7.89it/s]


In [11]:
!python ./createSubmissionFile.py -s "{nnUNet_results}{dataset_name}/predTs" -d "{nnUNet_results}/predTs_rename"
!(cd {nnUNet_results}) && (zip pred.zip "predTs_rename")

[01/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_168.nii.gz to /kaggle/working//predTs_rename/68.nii.gz
[02/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_239.nii.gz to /kaggle/working//predTs_rename/39.nii.gz
[03/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_186.nii.gz to /kaggle/working//predTs_rename/86.nii.gz
[04/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_071.nii.gz to /kaggle/working//predTs_rename/71.nii.gz
[05/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_256.nii.gz to /kaggle/working//predTs_rename/56.nii.gz
[06/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_005.nii.gz to /kaggle/working//predTs_rename/05.nii.gz
[07/50] Convert file from /kaggle/working/Dataset004_Hippocampus/predTs/hippocampus_254.nii.gz to /kaggle/working//predTs_rename/54.nii.gz
[08/50] Convert file from /